In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import jarque_bera, shapiro, durbin_watson
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("📊 MODEL DIAGNOSTICS TOOLKIT LOADED")
print("=" * 40)
print("✅ Statistical tests: VIF, significance, residuals")
print("✅ Diagnostic plots: Q-Q, residual analysis")
print("✅ Model assumptions validation")


ImportError: cannot import name 'durbin_watson' from 'scipy.stats' (/Users/nikmarf/icetest2/Ice-Cream/.venv/lib/python3.12/site-packages/scipy/stats/__init__.py)

In [ ]:
# Load the processed data (adjust path as needed)
try:
    # Try to load from the same source as other notebooks
    data = pd.read_csv('../data/processed/unified_data_with_weather.csv')
    print("✅ Data loaded successfully")
except:
    print("⚠️ Please adjust the data path to match your file structure")
    # Alternative: Create sample data for demonstration
    np.random.seed(42)
    n_weeks = 120
    data = pd.DataFrame({
        'sales': np.random.normal(135000, 25000, n_weeks),
        'tv_adstock': np.random.normal(5000, 1500, n_weeks),
        'digital_adstock': np.random.normal(3000, 800, n_weeks),
        'ooh_adstock': np.random.normal(1500, 500, n_weeks),
        'print_adstock': np.random.normal(800, 300, n_weeks),
        'in_store_promo': np.random.binomial(1, 0.3, n_weeks),
        'temperature': np.random.normal(12, 8, n_weeks),
        'holiday_flag': np.random.binomial(1, 0.1, n_weeks),
        'month_sin': np.sin(2 * np.pi * np.arange(n_weeks) / 52),
        'month_cos': np.cos(2 * np.pi * np.arange(n_weeks) / 52)
    })
    print("📝 Using simulated data for demonstration")

print(f"\n📊 Dataset Overview:")
print(f"   Observations: {len(data)}")
print(f"   Features: {data.shape[1]}")
print(f"   Missing values: {data.isnull().sum().sum()}")


In [ ]:
# Define feature sets (adjust based on your actual features)
media_features = [col for col in data.columns if '_adstock' in col]
control_features = ['in_store_promo', 'temperature', 'holiday_flag', 'month_sin', 'month_cos']
control_features = [f for f in control_features if f in data.columns]

all_features = media_features + control_features
target = 'sales'

print(f"\n🎯 Model Features:")
print(f"   Media channels: {media_features}")
print(f"   Control variables: {control_features}")
print(f"   Total features: {len(all_features)}")

# Prepare feature matrix and target
X = data[all_features].fillna(0)
y = data[target]

print(f"\n✅ Data prepared for diagnostics")


In [ ]:
def calculate_vif(X):
    """Calculate VIF for all features"""
    # Add constant for VIF calculation
    X_with_const = sm.add_constant(X)
    
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i+1) 
                       for i in range(len(X.columns))]
    
    # Add status
    vif_data['Status'] = vif_data['VIF'].apply(
        lambda x: '✅ Low' if x < 5 else '⚠️ Moderate' if x < 10 else '❌ High'
    )
    
    return vif_data.sort_values('VIF', ascending=False)

# Calculate VIF
print("🔍 MULTICOLLINEARITY ANALYSIS (VIF)")
print("=" * 40)

vif_results = calculate_vif(X)
print(vif_results.to_string(index=False))

# Summary
high_vif = vif_results[vif_results['VIF'] > 10]
moderate_vif = vif_results[(vif_results['VIF'] >= 5) & (vif_results['VIF'] <= 10)]
low_vif = vif_results[vif_results['VIF'] < 5]

print(f"\n📊 VIF Summary:")
print(f"   ✅ Low multicollinearity: {len(low_vif)} features")
print(f"   ⚠️ Moderate multicollinearity: {len(moderate_vif)} features")
print(f"   ❌ High multicollinearity: {len(high_vif)} features")

if len(high_vif) > 0:
    print(f"\n⚠️ High VIF features to consider removing:")
    for _, row in high_vif.iterrows():
        print(f"   - {row['Feature']}: VIF = {row['VIF']:.2f}")
